In [1]:
from dash import Dash, html, dcc , callback, Input, Output
from jupyter_dash import JupyterDash
import dash
import plotly.express as px
import pandas as pd

In [5]:
df = pd.json_normalize(pd.read_json('/Users/sugengw07/Documents/PROYEK/AlcoDash/ppmpkm_2023_202304160724.json', 
                                    orient='records')['ppmpkm_2023'])

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269905 entries, 0 to 269904
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   NAMA           269905 non-null  object 
 1   KDMAP          269905 non-null  object 
 2   KDBAYAR        269905 non-null  object 
 3   MASA           269905 non-null  object 
 4   MASA2          269905 non-null  object 
 5   TAHUN          269905 non-null  int64  
 6   TANGGALBAYAR   269905 non-null  int64  
 7   BULANBAYAR     269905 non-null  int64  
 8   TAHUNBAYAR     269905 non-null  int64  
 9   DATEBAYAR      269905 non-null  object 
 10  NOMINAL        269905 non-null  float64
 11  NTPN           269905 non-null  object 
 12  KET            269905 non-null  object 
 13  NPWP           269905 non-null  object 
 14  KPP            269905 non-null  object 
 15  NAMA_WP        269905 non-null  object 
 16  NAMA_AR        269905 non-null  object 
 17  SEKSI          269905 non-nul

In [57]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1('Testing by gengsu'),
    dcc.Dropdown(df['NM_KATEGORI'].unique(),id='sektor'),
    dcc.Graph(id='monthly_grap', figure={})
    ])

@callback(Output(component_id='monthly_grap', component_property='figure'),
          Input(component_id='sektor', component_property='value'))
def filtersektor(sektor):
    if not sektor:
        raise dash.exceptions.PreventUpdate
    else:
        sektor = df[(df['NM_KATEGORI']==sektor) & (df['TAHUNBAYAR']==2023)]
        sektor = sektor.groupby('BULANBAYAR')['NOMINAL'].sum().reset_index()
        bar = px.bar(sektor, x='NOMINAL', y='BULANBAYAR', orientation='h', title='PerSektor per bulan')
    return bar




if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')

Dash is running on http://127.0.0.1:8050/



In [7]:
dict_sektor = {
    'PERDAGANGAN BESAR DAN ECERAN; REPARASI DAN PERAWATAN MOBIL DAN SEPEDA MOTOR': 'PERDAGANGAN BESAR ECERAN<br>REPARASI PERAWATAN MOBIL',
    'PENYEDIAAN AKOMODASI DAN PENYEDIAAN MAKAN MINUM': 'PENYEDIAAN AKOMODASI<br>DAN PENYEDIAAN MAKAN MINUM',
    "PENGADAAN LISTRIK, GAS,UAP/AIR PANAS DAN UDARA DINGIN": "PENGADAAN LISTRIK, GAS,UAP/AIR PANAS<br>UDARA DINGIN",
    "PENGADAAN AIR, PENGELOLAAN SAMPAH DAN DAUR ULANG, PEMBUANGAN DAN PEMBERSIHAN LIMBAH DAN SAMPAH": "PENGADAAN AIR, PENGELOLAAN SAMPAH",
    "KEGIATAN BADAN INTERNASIONAL DAN BADAN EKSTRA INTERNASIONAL LAINNYA": "KEGIATAN BADAN INTERNASIONAL",
    "JASA PERSEWAAN, KETENAGAKERJAAN, AGEN PERJALANAN DAN PENUNJANG USAHA LAINNYA": "JASA PERSEWAAN, KETENAGAKERJAAN",
    "JASA PERORANGAN YANG MELAYANI RUMAH TANGGA; KEGIATAN YANG MENGHASILKAN BARANG DAN JASA OLEH RUMAH TANGGA YANG DIGUNAKAN SENDIRI UNTUK MEMENUHI KEBUTUHAN": "JASA PERORANGAN MELAYANI<br>RUMAH TANGGA",
    "ADMINISTRASI PEMERINTAHAN DAN JAMINAN SOSIAL WAJIB": "ADMINISTRASI PEMERINTAHAN<br>JAMINAN SOSIAL WAJIB",
    "JASA PENDIDIKAN": "JASA PENDIDIKAN",
    "INFORMASI DAN KOMUNIKASI":"INFORMASI KOMUNIKASI",
    "TRANSPORTASI DAN PERGUDANGAN" : "TRANSPORTASI DAN PERGUDANGAN",
    "JASA KESEHATAN DAN KEGIATAN SOSIAL": "JASA KESEHATAN DAN KEGIATAN SOSIAL",
    "KONSTRUKSI": "KONSTRUKSI",
    "REAL ESTAT" : "REAL ESTAT",
    "KEGIATAN JASA LAINNYA": "KEGIATAN JASA LAINNYA",
    "INDUSTRI PENGOLAHAN" : "INDUSTRI PENGOLAHAN",
    "KLU ERROR":"KLU ERROR",
    "PERTAMBANGAN DAN PENGGALIAN" : "PERTAMBANGAN DAN PENGGALIAN",
    "JASA KEUANGAN DAN ASURANSI" :"JASA KEUANGAN DAN ASURANSI",
    "PERTANIAN, KEHUTANAN DAN PERIKANAN" :"PERTANIAN,KEHUTANAN,PERIKANAN",
    "JASA PROFESIONAL, ILMIAH DAN TEKNIS":"JASA PROFESIONAL,ILMIAH,TEKNIS"
}

### Set Default Color Themes

In [23]:
px.defaults.color_continuous_scale = px.colors.sequential.Darkmint

In [36]:
px.bar(df
    .query("TAHUNBAYAR == 2023 & KET not in ('SPMKP')")
    .groupby('NM_KATEGORI', as_index=False)['NOMINAL']
    .sum()
    .sort_values(by='NOMINAL', ascending=True)[:10]
    .replace(dict_sektor)
    .assign(milyar = lambda x:(x['NOMINAL']/1000000000).apply(lambda y:'{:.2f}M'.format(y))),
    x='milyar',
    y='NM_KATEGORI',
    orientation='h',
    height=720,
    title='Penerimaan Per Sektor 2023 (bruto)',
    labels={'NM_KATEGORI':'', 'NOMINAL':''},
    color='NOMINAL',
    # color_continuous_scale='ylgnbu',
    text = 'milyar'
    # text_auto=',.2f'
).update_layout(
    title = dict(x=0.5, y=0.9),
    coloraxis_showscale=False,
    plot_bgcolor='white',
    xaxis_visible=False,
    margin = dict(l=0)
    )

### Analyze Penerimaan by Day

In [50]:
px.scatter(df
    .query("TAHUNBAYAR == 2023 & KET not in ('SPMKP')")
    .groupby(['TANGGALBAYAR','MAP'], as_index=False)['NOMINAL']
    .sum(),
    x='TANGGALBAYAR',
    y='NOMINAL',
    color='MAP',
    trendline='ols',
    trendline_scope='overall',
    marginal_x='histogram',
    marginal_y='violin'
)